    ПРОЕКТ 
    Мини-проект по Airflow будет проверяться нами вручную, поэтому на этом шаге - полное описание всех этапов его выполнения.

    1. Прочитайте csv файл, который находится по этой ссылке, при помощи библиотеки pandas

        https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ti6Su94955DZ4Tky8EbwifpgZf_dTjpBdiVH0Ukhsq94jZdqoHuUytZsFZKfwpXEUCKRFteJRc9P/pub?gid=889004448&single=true&output=csv
    2. В данных вы найдете информацию о событиях, которые произошли с объявлением 121288 за два дня. Рассчитайте следующие метрики в разрезе каждого дня: количество показов, количество кликов, CTR, сумма потраченных денег. То есть для каждой метрики у вас должно получиться две цифры, за 2019-04-01 и 2019-04-02.

    Рассчитать сумму потраченных денег можно по следующей формуле:
    Значение из колонки ad_cost разделить на 1000 и умножить на количество показов объявления.

    3. Теперь найдите процентную разницу между этими метриками. То есть найдите, насколько процентов каждая метрика увеличилась/уменьшилась 2 апреля по сравнению с 1 апреля.

    4. Создайте текстовый файл, в котором будет собрана информация о том, какие метрики наблюдаются 2 апреля (представьте, что 2 апреля это сегодня, условно, вы отправляете отчет за сегодня, сравнивая данные со вчера), а также, на сколько процентов они уменьшились по сравнению со вчера. То есть в текстовом файле должен быть список метрик, их значения 2 апреля и процентное отличие от 1 апреля. Вариант текстового отчета:

    Отчет по объявлению 121288 за 2 апреля
    Траты: Х рублей (+Y%)
    Показы: X (+Y%)
    Клики: X (-Y%)
    CTR: X (-Y%)

    5. Отправьте получившийся текст к себе в личные сообщения во ВКонтакте в виде сообщения со сводкой метрик. 

    6. Все предыдущие шаги оформите в виде исполняемого скрипта (скриптов) и скрипта для DAG-а, в котором при помощи BashOperator-а (или других операторов на ваше усмотрение) будет вызываться исполняемый скрипт (скрипты).

    7. В расписании для крона укажите каждый понедельник в 12 утра. Так, чтобы ваш скрипт с рассчетом дневных метрик отправлялся вам в личку каждый понедельник в 12 утра из Airflow.

    8. После того как убедитесь, что все работает, отправьте Merge Request с вашей папкой (которая должна быть названа вашем именем), в которой лежат все нужные скрипты, в мастер https://git.lab.karpov.courses/lab/airflow/-/blob/master/dags/. Перед этим удалите ваш токен из скрипта, мы вставим свой для проверки.

    9. Ссылку на MR прикрепите на следующем шаге.


In [ ]:
import pandas as pd
import os
import vk_api
import json
import random
import requests

### Откроем файл и произведем необходимые расчеты

In [4]:
df

,date,time,event,platform,ad_id,ad_cost_type,ad_cost
0,2019-04-01,2019-04-01 0:00:48,view,web,121288,CPM,187.4
1,2019-04-01,2019-04-01 0:04:41,view,ios,121288,CPM,187.4
2,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
3,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
4,2019-04-01,2019-04-01 0:08:46,view,ios,121288,CPM,187.4
...,...,...,...,...,...,...,...
602,2019-04-02,2019-04-01 22:39:16,click,android,121288,CPM,187.4
603,2019-04-02,2019-04-01 22:40:29,click,android,121288,CPM,187.4
604,2019-04-02,2019-04-01 22:40:31,click,web,121288,CPM,187.4
605,2019-04-02,2019-04-01 22:44:14,click,ios,121288,CPM,187.4


In [7]:
df_grp = df.groupby(['date', 'event'], as_index=False).agg({'time': 'count'})

In [8]:
df_grp

,date,event,time
0,2019-04-01,click,17
1,2019-04-01,view,491
2,2019-04-02,click,6
3,2019-04-02,view,93


In [3]:
df = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vR-ti6Su94955DZ4Tky8EbwifpgZf_dTjpBdiVH0Ukhsq94jZdqoHuUytZsFZKfwpXEUCKRFteJRc9P/pub?gid=889004448&single=true&output=csv')

In [14]:
df_grp_pivot = df_grp.pivot(index='date', columns='event', values='time').reset_index()

In [15]:
df_grp_pivot

event,date,click,view
0,2019-04-01,17,491
1,2019-04-02,6,93


In [16]:
df_grp_pivot['ctr'] = df_grp_pivot['click'] / df_grp_pivot['view']

In [40]:
df_grp_pivot

event,date,click,view,ctr
0,2019-04-01,17,491,0.034623
1,2019-04-02,6,93,0.064516


In [38]:
(6-17)/17*100

-64.70588235294117

In [ ]:
#df_grp_pivot['ctr'] = df_grp_pivot['click'] / df_grp_pivot['view']

In [50]:
df_grp_pivot.loc[df_grp_pivot.index == 0,'click'].values[0]

17

In [51]:
df_grp_pivot.loc[df_grp_pivot.index == 1,'click'].values[0]

6

In [61]:
df_click_percent_var = (df_grp_pivot.loc[df_grp_pivot.index == 1,'click'].values[0] \
               - df_grp_pivot.loc[df_grp_pivot.index == 0,'click'].values[0]) \
                / df_grp_pivot.loc[df_grp_pivot.index == 0,'click'].values[0] \
                * 100

In [62]:
df_view_percent_var = (df_grp_pivot.loc[df_grp_pivot.index == 1,'view'].values[0] \
               - df_grp_pivot.loc[df_grp_pivot.index == 0,'view'].values[0]) \
                / df_grp_pivot.loc[df_grp_pivot.index == 0,'view'].values[0] \
                * 100

In [63]:
df_ctr_percent_var = (df_grp_pivot.loc[df_grp_pivot.index == 1,'ctr'].values[0] \
               - df_grp_pivot.loc[df_grp_pivot.index == 0,'ctr'].values[0]) \
                / df_grp_pivot.loc[df_grp_pivot.index == 0,'ctr'].values[0] \
                * 100

In [64]:
df_grp_pivot

event,date,click,view,ctr,click_percent,view_percent,ctr_percent
0,2019-04-01,17,491,0.034623,-64.705882,-81.059063,86.337761
1,2019-04-02,6,93,0.064516,-64.705882,-81.059063,86.337761


In [19]:
df

,date,time,event,platform,ad_id,ad_cost_type,ad_cost
0,2019-04-01,2019-04-01 0:00:48,view,web,121288,CPM,187.4
1,2019-04-01,2019-04-01 0:04:41,view,ios,121288,CPM,187.4
2,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
3,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
4,2019-04-01,2019-04-01 0:08:46,view,ios,121288,CPM,187.4
...,...,...,...,...,...,...,...
602,2019-04-02,2019-04-01 22:39:16,click,android,121288,CPM,187.4
603,2019-04-02,2019-04-01 22:40:29,click,android,121288,CPM,187.4
604,2019-04-02,2019-04-01 22:40:31,click,web,121288,CPM,187.4
605,2019-04-02,2019-04-01 22:44:14,click,ios,121288,CPM,187.4


In [26]:
df_cost = df.groupby(['date', 'event']).agg({'ad_cost': 'sum', 'event': 'count'}) \
            .rename(columns={'event' : 'event_count'}) \
            .reset_index()

In [27]:
df_cost

,date,event,ad_cost,event_count
0,2019-04-01,click,3185.8,17
1,2019-04-01,view,92013.4,491
2,2019-04-02,click,1124.4,6
3,2019-04-02,view,17428.2,93


In [29]:
df_cost['general_cost'] = df_cost['ad_cost'] / 1000 * df_cost['event_count']

In [30]:
df_cost

,date,event,ad_cost,event_count,general_cost
0,2019-04-01,click,3185.8,17,54.1586
1,2019-04-01,view,92013.4,491,45178.5794
2,2019-04-02,click,1124.4,6,6.7464
3,2019-04-02,view,17428.2,93,1620.8226


In [58]:
df_cost_var = (df_cost.loc[df_cost.index == 3,'general_cost'].values[0] \
               - df_cost.loc[df_cost.index == 1,'general_cost'].values[0]) \
                / df_cost.loc[df_cost.index == 1,'general_cost'].values[0] \
                * 100

In [59]:
df_cost_var

-96.41240910731246

### Напишем текстовый файл-отчет с процентным соотношением динамики метрик

In [94]:
string_to_file = 'Отчет по объявлению 121288 за 2 апреля\n\
Траты: {} рублей ({}%)\n\
Показы: {} ({}%)\n\
Клики: {} ({}%)\n\
CTR: {} ({}%)'

In [116]:
print(string_to_file.format(df_cost.loc[df_cost.index == 3,'general_cost'].values[0]\
, df_cost_var\
, df_cost.loc[df_cost.index == 3,'event_count'].values[0]\
, df_view_percent_var\
, df_cost.loc[df_cost.index == 2,'event_count'].values[0]\
, df_click_percent_var\
, df_grp_pivot.loc[df_grp_pivot.index == 1,'ctr'].values[0]\
, df_ctr_percent_var\
))

Отчет по объявлению 121288 за 2 апреля
Траты: 1620.8225999999986 рублей (-96.41240910731246%)
Показы: 93 (-81.05906313645622%)
Клики: 6 (-64.70588235294117%)
CTR: 0.06451612903225806 (86.33776091081593%)


### Запишем в текстовый файл

In [104]:
file_loc = open('/home/jupyter-m.mandrik-1/20_3_miniproject_metrics.txt', 'w')

In [105]:
file_loc.write(string_to_file.format(df_cost.loc[df_cost.index == 3,'general_cost'].values[0].round(0)\
, df_cost_var.round(0)\
, df_cost.loc[df_cost.index == 3,'event_count'].values[0].round(0)\
, df_view_percent_var.round(0)\
, df_cost.loc[df_cost.index == 2,'event_count'].values[0].round(0)\
, df_click_percent_var.round(0)\
, df_grp_pivot.loc[df_grp_pivot.index == 1,'ctr'].values[0]\
, df_ctr_percent_var.round(0)\
))

139

In [106]:
file_loc.close()

### Создадим запрос в VK


In [112]:
app_token = '0dc3a8c29818b5f1924c29966c93ff146300cfdb1ddbdf69b2f3830b08fdebf647afbcfa3a4094aec58ab'
chat_id = 1
my_id = 558437518
vk_session = vk_api.VkApi(token=app_token)
vk = vk_session.get_api()

In [115]:
path_to_file = '/home/jupyter-m.mandrik-1/20_3_miniproject_metrics.txt'
file_name = '20_3_miniproject_metrics.txt'

upload_url = vk.docs.getMessagesUploadServer(peer_id=my_id)["upload_url"]
file = {'file': (file_name, open(path_to_file, 'rb'))}

response = requests.post(upload_url, files=file)

json_data = json.loads(response.text)

saved_file = vk.docs.save(file=json_data['file'], title=file_name)
attachment = 'doc{}_{}'.format(saved_file['doc']['owner_id'], saved_file['doc']['id'])

vk.messages.send(
    chat_id=chat_id,
    random_id=random.randint(1, 2 ** 31),
    message='Минипроект 20-3',
    attachment=attachment)


5